## Import Important Libraries

In [20]:
import torch
import torchvision
from torchvision import transforms

## Write class for Nearest Neighbor Classification Algorithm

In [21]:
class NearestNeighbor(object):
  def __init__(self):
    pass
  
  def train(self, x, y):
    self.x_train = x
    self.y_train = y

  def predict(self, x):
    # get the batchsize
    b = x.shape[0]
    # build an array of zeros with sape same as number of samples in testing data
    # and put it on cuda (if available)
    y_pred = torch.zeros(b, dtype=self.y_train.dtype).to(device)
    # Go through each image sample in training data and calculate L1 distance
    # between that single image from testing set and all images from the training set
    for i in range(b):
      # calculating distances
      distances = torch.sum(torch.abs(self.x_train - x[i, :]), axis=1)
      # get the nearest image i.e minimun distance
      min_idx = torch.argmin(distances)
      # save the prediction
      y_pred[i] = self.y_train[min_idx]
    return y_pred

## Download dataset, apply transforms, and build a dataloader

In [22]:
# originally images from torchvision datasets will be PILLOW images
# so we use transforms to convert it into tensor
ds_transforms = transforms.Compose([
    transforms.ToTensor()
])
# Check if the cuda is available then use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using: {device}')

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=ds_transforms,)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=ds_transforms)
train_bs = len(train_set)
test_bs = len(test_set)

# Build dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_bs, shuffle=False)

Using: cuda
Files already downloaded and verified
Files already downloaded and verified


## Extract data from the dataloader as single batch
then reshape it, and move it to gpu device

In [23]:
# because our batch size is same is total length of the dataset
# we just use next(iter(loader)) to get the first batch which in this case will load all training examples
train_data, train_labels = next(iter(train_loader))
# reshape the data in (b, 3*32*32) and then put it on available device (cuda or cpu)
train_data, train_labels = train_data.reshape(train_data.shape[0], -1).to(device), train_labels.reshape(train_labels.shape[0], -1).to(device)

In [24]:
train_data.shape

torch.Size([50000, 3072])

In [25]:
test_data, test_labels = next(iter(test_loader))
test_data, test_labels = test_data.reshape(test_data.shape[0], -1).to(device), test_labels.reshape(test_data.shape[0], -1).to(device)

In [26]:
test_data.shape

torch.Size([10000, 3072])

## Train nearest neighbor algorithm
Here we are just storing the training data and not performing any real-training.

Then predict the labels for testing dataset. This involves calculating L1 distances between each pair of testing dataset against all images in training set

In [27]:
# create an instance of our NearestNeighbor() class 
nn = NearestNeighbor()
# Feed Training data (it will just save it in the memory and do nothing)
nn.train(train_data, train_labels)
# go through every testing sample and calcualte distances between that sample and all the training images
y_pred = nn.predict(test_data)


In [61]:
test_labels.shape, y_pred.shape

(torch.Size([10000, 1]), torch.Size([10000]))

## Unsqueese predicted labels for getting results

In [64]:
y_pred_reshaped = y_pred.unsqueeze(1)

In [65]:
y_pred_reshaped.shape

torch.Size([10000, 1])

# Calculate accuracy percentage

In [72]:
# get results in a human readible format
# y_pred_reshaped == test_labels will produce a boolean array of which predicted labels match
# to those of original labels
# .sum will get sum of all True boolean values
# .item() just extacts value of a tensor
result = (y_pred_reshaped == test_labels).sum().item()

In [74]:
print(f'Accuracy: {(result / test_labels.shape[0]) * 100}')

Accuracy: 38.59
